Firstly, mount your Google Drive to this notebook. Thus, a data file we generate and write will be recorded your Drive.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


These methods call Twitter API endpoints(URLs)

In [ ]:
import requests
import os
import json

# To set your environment variables in your terminal run the following line:
# export 'BEARER_TOKEN'='<your_bearer_token>'
bearer_token = 'AAAAAAAAAAAAAAAAAAAAACpYhgEAAAAASnPJKHL8giYbmnqq56S12XKkEyc%3DOZ8CJRFKirFMb2RU7kO68cwi4b2dOzhNdZ5yBSpR810Y0ykYYH'


def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """

    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2FilteredStreamPython"
    return r


# Retrieves existing rules we defined previously
def get_rules():
    response = requests.get(
        "https://api.twitter.com/2/tweets/search/stream/rules", auth=bearer_oauth
    )
    if response.status_code != 200:
        raise Exception(
            "Cannot get rules (HTTP {}): {}".format(response.status_code, response.text)
        )
    print(json.dumps(response.json()))
    return response.json()

# Deletes all previously defined rules
def delete_all_rules(rules):
    if rules is None or "data" not in rules:
        return None

    ids = list(map(lambda rule: rule["id"], rules["data"]))
    payload = {"delete": {"ids": ids}}
    response = requests.post(
        "https://api.twitter.com/2/tweets/search/stream/rules",
        auth=bearer_oauth,
        json=payload
    )
    if response.status_code != 200:
        raise Exception(
            "Cannot delete rules (HTTP {}): {}".format(
                response.status_code, response.text
            )
        )
    print(json.dumps(response.json()))

# Set new rules for our own stream. If you don't delete previous
# rules first, it sets the combination of old and new rules
# Rules can be considered as conditionals to filter data
# All rules explained
# https://developer.twitter.com/en/docs/twitter-api/tweets/filtered-stream/integrate/build-a-rule

def set_rules():
    ###### DEFINE RULES ################
    ####################################
    ####################################
    ####################################
    sample_rules = [
        {"value": "#cdnpoli -has:links -has:media -has:images"}
    ]
    payload = {"add": sample_rules}
    response = requests.post(
        "https://api.twitter.com/2/tweets/search/stream/rules", 
        auth=bearer_oauth,
        json=payload,
    )
    if response.status_code != 201:
        raise Exception(
            "Cannot add rules (HTTP {}): {}".format(response.status_code, response.text)
        )
    print(json.dumps(response.json()))

Delete old rules

In [ ]:
old_rules = get_rules()
delete = delete_all_rules(old_rules)

{"data": [{"id": "1581032618858106880", "value": "#cdnpoli -has:links -has:media -has:images"}], "meta": {"sent": "2022-10-28T17:13:11.508Z", "result_count": 1}}
{"meta": {"sent": "2022-10-28T17:13:12.730Z", "summary": {"deleted": 1, "not_deleted": 0}}}


In [ ]:
rules = set_rules()

{"data": [{"value": "#cdnpoli -has:links -has:media -has:images", "id": "1586043428391063553"}], "meta": {"sent": "2022-10-28T17:13:16.663Z", "summary": {"created": 1, "not_created": 0, "valid": 1, "invalid": 0}}}


In [ ]:
# Return the tweets as json format according to defined rules
def get_stream_write_file(set):


  # How to add query parameters to the api url : https://developer.twitter.com/en/docs/twitter-api/tweets/filtered-stream/quick-start
    response = requests.get(
        "https://api.twitter.com/2/tweets/search/stream?tweet.fields=created_at", auth=bearer_oauth, stream=True,
    )
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(
            "Cannot get stream (HTTP {}): {}".format(
                response.status_code, response.text
            )
        )
    with open('/content/drive/My Drive/tweets_cdnpoli_Jan.txt', 'a') as f:
      for response_line in response.iter_lines():
        if response_line:
          json_response = json.loads(response_line)
          json_str = json.dumps(json_response, indent=4, sort_keys=True)
          f.write(json_str)

In [ ]:
get_stream_write_file(rules)

In [ ]:
drive.flush_and_unmount()